### Import Libraries and modules

In [1]:
import pandas as pd
import numpy as np
import requests
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

Getting data from Coingecko API

In [2]:
def get_token_data(token_ids):
    url = "https://api.coingecko.com/api/v3/coins/markets"
    params = {
        "vs_currency": "usd",
        "ids": ",".join(token_ids),
        "price_change_percentage": "1h,24h,7d,30d,1y"
    }
    response = requests.get(url, params=params)
    data = response.json()
    df = pd.DataFrame(data)
    
    cols = ["id", "symbol", "current_price", "ath", "market_cap_rank", "price_change_percentage_30d_in_currency", "price_change_percentage_1y_in_currency"]
    cols = [c for c in cols if c in df.columns] 
    df = df[cols]
    df.rename(columns={"price_change_percentage_30d_in_currency": "price_change_30d", "price_change_percentage_1y_in_currency" : "price_change_1y"}, inplace=True)
    return df


tokens = ["bitcoin", "ethereum", "solana", "cardano", "dogecoin", "shiba-inu", "pepe", "kadena"]

df = get_token_data(tokens)
print("\n📊 Raw data:\n", df.head())



📊 Raw data:
          id symbol  current_price            ath  market_cap_rank  \
0   bitcoin    btc  111539.000000  126080.000000                1   
1  ethereum    eth    3953.830000    4946.050000                2   
2    solana    sol     194.450000     293.310000                6   
3  dogecoin   doge       0.197334       0.731578                9   
4   cardano    ada       0.658227       3.090000               11   

   price_change_30d  price_change_1y  
0          1.881845        66.468666  
1          0.849506        59.388910  
2         -1.888722        14.916107  
3        -12.873022        46.999756  
4        -15.073729        95.960309  


Create features

In [3]:
df["price_drop_from_ath"] = (df["ath"] - df["current_price"]) / df["ath"] * 100
df["is_strong"] = (df["price_change_1y"] > 0).astype(int)

features = df[["price_drop_from_ath", "price_change_30d", "price_change_1y"]]
labels = df["is_strong"]

Train Model

In [4]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.3, random_state=42)

model = RandomForestClassifier(n_estimators=50, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print("✅ Model evaluation:\n")
print(classification_report(
    y_test,
    y_pred,
    labels=[0, 1],
    target_names=["Weak", "Strong"],
    zero_division=0
))



✅ Model evaluation:

              precision    recall  f1-score   support

        Weak       0.00      0.00      0.00         1
      Strong       0.67      1.00      0.80         2

    accuracy                           0.67         3
   macro avg       0.33      0.50      0.40         3
weighted avg       0.44      0.67      0.53         3



Predict

In [5]:
proba = model.predict_proba(features)

if proba.shape[1] == 1:
    df["strength_score"] = proba[:, 0]
    df["Health"] = np.where(df["strength_score"] > 0.5, "🟢 Strong", "🔴 Weak")
else:
    df["strength_score"] = proba[:, 1]
    df["Health"] = np.where(df["strength_score"] > 0.5, "🟢 Strong", "🔴 Weak")

print("\n🌐 Token Strength Report:\n")
print(df[["symbol", "current_price", "price_drop_from_ath", "price_change_30d", "Health"]])



🌐 Token Strength Report:

  symbol  current_price  price_drop_from_ath  price_change_30d    Health
0    btc  111539.000000            11.533154          1.881845  🟢 Strong
1    eth    3953.830000            20.060857          0.849506  🟢 Strong
2    sol     194.450000            33.704954         -1.888722  🟢 Strong
3   doge       0.197334            73.026253        -12.873022  🟢 Strong
4    ada       0.658227            78.698155        -15.073729  🟢 Strong
5   shib       0.000010            88.161560        -13.777962  🟢 Strong
6   pepe       0.000007            74.313236        -21.082262    🔴 Weak
7    kda       0.058444            99.788553        -82.373987    🔴 Weak
